In [27]:
import pymongo
import pandas as pd
import sqlalchemy

In [2]:
user = 'team04'
pw = '1111'
host = 'ec2-54-95-8-243.ap-northeast-1.compute.amazonaws.com'
client = pymongo.MongoClient(f'mongodb://{user}:{pw}@{host}:27017/')
db = client.resell

In [41]:
df = pd.DataFrame(db.trade.find({}, {'_id': 0, 'model_no': 1, 'trade': 1}))
df

,model_no,trade
0,CZ0790-100,"270\n208,000원\n22/07/12"
1,CZ0790-100,"270\n270,000원\n빠른배송\n22/07/07"
2,CZ0790-100,"270\n250,000원\n22/07/07"
3,CZ0790-100,"270\n269,000원\n빠른배송\n22/07/07"
4,CZ0790-100,"270\n255,000원\n22/07/05"
...,...,...
261923,555088-161,"235(US 5)\n245,000원\n22/04/26"
261924,555088-161,"235(US 5)\n239,000원\n22/04/26"
261925,555088-161,"235(US 5)\n245,000원\n22/04/20"
261926,555088-161,"235(US 5)\n247,000원\n22/04/16"


In [42]:
df['size'] = df['trade'].apply(lambda x: x.split('\n')[0].split('(')[0])
df['trade_date'] = df['trade'].apply(lambda x: x.split('\n')[2] if len(x.split('\n')) == 3 else x.split('\n')[3])
df['price'] = df['trade'].apply(lambda x: int(x.split('\n')[1].replace(',', '').replace('원', '')))
df.head()

,model_no,trade,size,trade_date,price
0,CZ0790-100,"270\n208,000원\n22/07/12",270,22/07/12,208000
1,CZ0790-100,"270\n270,000원\n빠른배송\n22/07/07",270,22/07/07,270000
2,CZ0790-100,"270\n250,000원\n22/07/07",270,22/07/07,250000
3,CZ0790-100,"270\n269,000원\n빠른배송\n22/07/07",270,22/07/07,269000
4,CZ0790-100,"270\n255,000원\n22/07/05",270,22/07/05,255000


In [43]:
df['trade_date'] = pd.to_datetime(df['trade_date'], format='%y/%m/%d')
df['size'] = df['size'].astype(int)

In [44]:
df_orderd = df[['model_no', 'size', 'trade_date', 'price']].sort_index(ascending=False)
df_orderd = df_orderd.reset_index(drop=True)
df_orderd

,model_no,size,trade_date,price
0,555088-161,235,2022-04-08,240000
1,555088-161,235,2022-04-16,247000
2,555088-161,235,2022-04-20,245000
3,555088-161,235,2022-04-26,239000
4,555088-161,235,2022-04-26,245000
...,...,...,...,...
261923,CZ0790-100,270,2022-07-05,255000
261924,CZ0790-100,270,2022-07-07,269000
261925,CZ0790-100,270,2022-07-07,250000
261926,CZ0790-100,270,2022-07-07,270000


In [45]:
df_orderd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261928 entries, 0 to 261927
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   model_no    261928 non-null  object        
 1   size        261928 non-null  int32         
 2   trade_date  261928 non-null  datetime64[ns]
 3   price       261928 non-null  int64         
dtypes: datetime64[ns](1), int32(1), int64(1), object(1)
memory usage: 7.0+ MB


In [46]:
conn = f"mysql://{user}:{pw}@{host}/resell?charset=utf8"
engine = sqlalchemy.create_engine(conn, encoding='utf-8')

In [47]:
df_orderd.to_sql(
    'trade_history',
    con=engine,
    if_exists='append',
    index=False,
    dtype={
        'model_no': sqlalchemy.types.String(40),
        'size': sqlalchemy.types.Integer,
        'trade_date': sqlalchemy.types.Date,
        'price': sqlalchemy.types.Integer
    }
)

261928